In [1]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
from ipywidgets import interact

# Database
conn = sqlite3.connect(':memory:')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS notes (id INTEGER PRIMARY KEY, title TEXT, content TEXT)''')
conn.commit()

# Tkinter setup
root = tk.Tk()
root.configure(bg="#87CEEB")
root.title("Sticky Notes!")

# Functions
def create_table():
    c.execute('''CREATE TABLE IF NOT EXISTS notes (id INTEGER PRIMARY KEY, title TEXT, content TEXT)''')
    conn.commit()

def add_note(title, content):
    c.execute('''INSERT INTO notes (title, content) VALUES (?, ?)''', (title, content))
    conn.commit()
    messagebox.showinfo("Note Created", "Note added successfully!")

def view_notes():
    notes_list.delete(0, tk.END)
    for row in c.execute('''SELECT * FROM notes'''):
        notes_list.insert(tk.END, f"{row[1]} - {row[2]}")

def delete_note():
    selected_note = notes_list.curselection()
    if selected_note:
        note_id = c.execute('''SELECT id FROM notes LIMIT 1 OFFSET ?''', (selected_note[0],)).fetchone()[0]
        c.execute('''DELETE FROM notes WHERE id = ?''', (note_id,))
        conn.commit()
        messagebox.showinfo("Note Deleted", "Note deleted successfully!")
        view_notes()

        # Clear the text in the Title and Content fields
        title_entry.delete(0, tk.END)
        content_text.delete("1.0", tk.END)
        
# GUI
title_label = tk.Label(root, text="Title:")
title_label.pack(side="top", pady=10)
title_label.config(bg='#FFC0CB') 

title_entry = tk.Entry(root, width=50)
title_entry.pack()

content_label = tk.Label(root, text="Content:")
content_label.pack(side="top", pady=10)
content_label.config(bg='#FFC0CB') 

content_text = tk.Text(root, wrap=tk.WORD, width=50, height=10)
content_text.pack()

add_button = tk.Button(root, text="Add Note", command=lambda: add_note(title_entry.get(), content_text.get("1.0", tk.END)))
add_button.pack(side="top", pady=10)
add_button.config(bg='#FDFD96') 

view_button = tk.Button(root, text="View Notes", command=view_notes)
view_button.pack(side="top", pady=10)
view_button.config(bg='#FDFD96') 

delete_button = tk.Button(root, text="Delete Note", command=delete_note)
delete_button.pack(side="top", pady=10)
delete_button.config(bg='#FDFD96') 

notes_list = tk.Label(root, text="Saved notes:")
notes_list.pack(side="top",  pady=10)
notes_list.config(bg='#FFC0CB') 
   
notes_list = tk.Listbox(root, selectmode=tk.SINGLE, width=50)
notes_list.pack()

# Run the Tkinter event loop
root.mainloop()
